In [2]:
import re

headerlines = []

worktext = {}

bookrefs = []

firstpages = {}

firstpagenums = {}

printvols = {
    '1' : 'https://archive.org/details/lucianha01luciuoft/page/PNUM/mode/2up',
    '2' : 'https://archive.org/details/lucianhar02luciuoft/page/PNUM/mode/2up',
    '3' : 'https://archive.org/details/lucianhar03luciuoft/page/PNUM/mode/2up',
    '4' : 'https://archive.org/details/lucianhar04luciuoft/page/PNUM/mode/2up',
    '5' : 'https://archive.org/details/lucianhar05luciuoft/page/PNUM/mode/2up'

}

hathtipageoffsets = {
    1 : 12,
    2 : 14,
    3 : 12,
    4 : 8,
    5 : 12
    
}

f = open('bookrefs.txt')
for l in f:
    l = re.sub('\s+$','',l)
    bookrefs.append(l)

f.close()

commonw = ['an','a','at','for','in','it','of','on','not','or','the','to']
caplist = ['I','II']
def setcase(s):
    res = []
    if(s in caplist):
        return(s)
    for foo in s.split():
        foo = foo.lower()
        if(not foo in commonw):
    
            foo = re.sub('^.',foo[0].upper(),foo)
        res.append(foo)
    news =  ' '.join(res)
    news = re.sub('^.',news[0].upper(),news)
    return(news)

def buildmetafile(curwork,uselabel):        #now build the metadata file
        return # already built!
        fname = '/Users/gcrane/github/canonical-greekLit/data/' + re.sub('\.','/',curwork) + '/__cts__.xml'
        f = open(fname)
        
        ctslines = []
        basedesc = ''
        for l in f:
            l = re.sub('\s+$','',l)
            m = re.search('<ti:descr',l)
            if(m):
                basedesc = l
                
            if(re.search('</ti:work>',l)):
                ctslines.append('   <ti:translation urn="urn:cts:greekLit:'+curwork+'.perseus-eng2" xml:lang="eng" workUrn="urn:cts:greekLit:'+curwork+'">')
                ctslines.append('      <ti:label xml:lang="eng">'+uselabel+'</ti:label>')
                ctslines.append(basedesc)
                ctslines.append('  </ti:translation>\n\n')
            ctslines.append(l)
        
        f.close()
        
        outfname = '/Users/gcrane/github/first1kwork/data/' + re.sub('\.','/',curwork) + '/__cts__.xml'
        outf = open(outfname,'w')
        for line in ctslines:
            print(line,file=outf)
        outf.close()

def splitworks(voln):
    global worktext
    worktext = {}
    
    fname = 'lucian-loeb0' + str(voln) + '.1st1k-eng1.xml'
    inbody = 0
    curwork = ''
    
    curhead = ''
    headerlines = []
    f = open(fname)
    for l in f:
        l = re.sub('\s+$','',l)
        if(re.search('<publisher>',l)):
            l = '<publisher>Trustees of Tufts University</publisher>'
        if(re.search('<name>Lisa Cerrato</name>',l)):
            l = '<name>Lisa Cerrato</name>\n<name>Gregory Crane</name>'
            
        if(not inbody):
            headerlines.append(l)
        if(re.search('</body>',l)):
            break
        if(re.search('<body>',l)):
            inbody = 1
        m = re.search('<div type="translation" n="(tlg[^"]+)"',l)
        if(m):
            curwork = m[1]
            #print(curwork)
            worktext[curwork] = []
        if(re.search('subtype="book"',l)):
            hasbooks = 1
        #if(not curwork):
        #    print(l)
        #else:
        if(curwork):
            worktext[curwork].append(l)
        m = re.search('<pb n="v\.([0-9])\.p\.([0-9]+)"',l)
        if(m and curwork and not curwork in firstpages):
            # always show Greek and English side by side. The Greek starts one page earlier
            firstpage = str(int(m[2]) - 1)
            firstpages[curwork] = re.sub('PNUM',firstpage,printvols[m[1]])
            firstpagenums[curwork] = firstpage
    f.close()
    
    for curwork in worktext:
        
        outfname2 = '/Users/gcrane/github/first1kwork/data/' + re.sub('\.','/',curwork) + '/' + curwork + '.perseus-eng2.xml'
        outf = open(outfname2,'w')
        curhead = ''
        usehead = ''
        inrefsdecl = 0
        hasbooks = 0
        baseurn = 'urn:cts:greekLit:' + curwork + '.perseus-eng2'
        
        for foo in worktext[curwork]:
            curbook = ''
            m = re.search('<head>\s*([^<]+)',foo)
            if(m and not usehead):
                usehead = setcase(m[1])
                #print(fname,curwork,curhead)
        for foo in headerlines:
            foo = re.sub('<editor>Austin Morris Harmon, 1878-1950','<editor role="translator">Austin Morris Harmon',foo)
            foo = re.sub('tlg0062\.tlg001',curwork,foo)
            if(re.search('^<title xml:lang="lat">',foo) and usehead):
                
                foo = '<title xml:lang="eng">' + usehead + '</title>'
                #print('newtit',foo)
            if( re.search('<refsDecl',foo)):
                inrefsdecl = 1
                if(hasbooks):
                    for boo in bookrefs:
                        print(boo,file=outf)
                    continue
                else:
                    print('<refsDecl n="CTS">\n<cRefPattern n="section" matchPattern="(\w+)" replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n=\'$1\'])">\n<p>This pointer pattern extracts section.</p>\n</cRefPattern>\n</refsDecl>',file=outf)
                    continue
            foo = re.sub('1st1K-eng1','perseus-eng2',foo)
            foo = re.sub('<resp>OCRd, tagged and added.+','<orgName>Tufts University</orgName>\n<resp>Digital conversion and editing</resp>',foo)
            if(re.search('</refsDecl>',foo) ):
                inrefsdecl = 0
                continue
            if(inrefsdecl ):
                continue
            if(re.search('<ref target="https://hdl.handle.net',foo)):
                print('<ref target="'+firstpages[curwork]+'">Internet Archive</ref>',file=outf)
                print('firstpage',curwork,firstpagenums[curwork])
                curpageseq = str(int(firstpagenums[curwork]) + hathtipageoffsets[voln])
# Hathi Trust URL: 
# https://hdl.handle.net/2027/osu.32435024202129?urlappend=%3Bseq=20
    
                foo = re.sub('(target="[^"]+)','\g<1>?urlappend=%3Bseq='+curpageseq,foo )
            print(foo,file=outf)
        for foo in worktext[curwork]:
            
            m = re.search('subtype="section" n="([^"]+)"',foo)
            if(m):
                curbook = ':' + m[1]
                foo = re.sub('subtype','xml:base="'+baseurn+'" subtype',foo)
            m = re.search('subtype="book" n="([^"]+)"',foo)
            if(m):
                print(baseurn)
                foo = re.sub('subtype','xml:base="'+baseurn+curbook+'" subtype',foo)
            m = re.search('<head>\s*([^<]+)',foo)
            if(m and not curhead):
                curhead = usehead
                foo = re.sub('<head>\s*[^<]+','<head>'+curhead,foo)
                #print('head',curwork,foo)
            foo = re.sub('<note','<note xml:lang="eng"',foo)
            foo = re.sub('<editor>','<editor role="translator">',foo)
            foo = re.sub('translation" n="(tlg0062.tlg[0-9]+)"','translation" n="urn:cts:greekLit:\g<1>.perseus-eng2" xml:lang="eng"',foo)
            print(foo,file=outf)
        print('</body></text></TEI>',file=outf)
        buildmetafile(curwork,usehead)
        
        
        
        outf.close()
        
for i in range(1,6):
    print(i)
    splitworks(i)


1
firstpage tlg0062.tlg001 4
urn:cts:greekLit:tlg0062.tlg001.perseus-eng2
urn:cts:greekLit:tlg0062.tlg001.perseus-eng2
firstpage tlg0062.tlg002 36
firstpage tlg0062.tlg003 48
firstpage tlg0062.tlg004 62
firstpage tlg0062.tlg005 74
firstpage tlg0062.tlg006 82
firstpage tlg0062.tlg007 98
firstpage tlg0062.tlg008 142
firstpage tlg0062.tlg009 176
firstpage tlg0062.tlg010 210
firstpage tlg0062.tlg011 222
firstpage tlg0062.tlg012 248
urn:cts:greekLit:tlg0062.tlg012.perseus-eng2
urn:cts:greekLit:tlg0062.tlg012.perseus-eng2
firstpage tlg0062.tlg013 360
firstpage tlg0062.tlg014 396
firstpage tlg0062.tlg015 412
2
firstpage tlg0062.tlg016 4
firstpage tlg0062.tlg017 60
firstpage tlg0062.tlg018 90
firstpage tlg0062.tlg019 172
firstpage tlg0062.tlg020 242
firstpage tlg0062.tlg021 268
firstpage tlg0062.tlg022 326
firstpage tlg0062.tlg023 396
firstpage tlg0062.tlg024 450
3
firstpage tlg0062.tlg025 4
firstpage tlg0062.tlg026 84
firstpage tlg0062.tlg027 154
firstpage tlg0062.tlg028 174
firstpage tlg0062

In [31]:
for foo in worktext:
    print(foo)

tlg0062.tlg001
tlg0062.tlg002
tlg0062.tlg003
tlg0062.tlg004
tlg0062.tlg005
tlg0062.tlg006
tlg0062.tlg007
tlg0062.tlg008
tlg0062.tlg009
tlg0062.tlg010
tlg0062.tlg011
tlg0062.tlg012
tlg0062.tlg013
tlg0062.tlg014
tlg0062.tlg015


In [69]:
worktext['tlg0062.tlg052']

['<div type="translation" n="tlg0062.tlg052">',
 '<head>DISOWNED<note><p>The fictitious case which underlies this declamation is',
 'outlined in the argument that precedes the text. An earlier',
 'treatment of the same theme is to be found in the Controversiae',
 'of Seneca Rhetor (IV, 5). In the Lucianic piece, the speaker’s',
 'references to his stepmother constitute a notable example of',
 'sustained irony as a rhetorical device.1</p></note></head>',
 '<div type="textpart" subtype="section" n="1"><p>',
 '',
 '',
 'A son who had been disowned studied medicine. When his',
 'father became insane and had been given up by the other',
 'doctors, he cured him by administering a remedy, and was again',
 'received into the family. After that, he was ordered to',
 'cure his stepmother, who was insane, and as he refused to do so,',
 'he is now being disowned again.<ref n="477.1"/>',
 'There is nothing novel or surprising, gentlemen of',
 'the jury, in my father’s present course, and this is',


In [49]:
s = 'abc'
s[0] = s[0].upper()
s

TypeError: 'str' object does not support item assignment